In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp v2.nixtla_client

# V2 Nixtla Client

In [ ]:
#| export
import logging
import math
import os
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import chain
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union

import httpx
import numpy as np
import pandas as pd
import utilsforecast.processing as ufp
from pydantic import NonNegativeInt, PositiveInt
from utilsforecast.compat import DataFrame
from utilsforecast.feature_engineering import _add_time_features, time_features
from utilsforecast.validation import validate_format, validate_freq

from nixtla.core.api_error import ApiError
from nixtla.core.http_client import HttpClient
from nixtla.utils import _restrict_input_samples

In [ ]:
#| export
logger = logging.getLogger(__name__)

In [ ]:
#| exporti
_LOSS = Literal["default", "mae", "mse", "rmse", "mape", "smape"]
_MODEL = Literal["timegpt-1", "timegpt-1-long-horizon"]

_date_features_by_freq = {
    # Daily frequencies
    'B': ['year', 'month', 'day', 'weekday'],
    'C': ['year', 'month', 'day', 'weekday'],
    'D': ['year', 'month', 'day', 'weekday'],
    # Weekly
    'W': ['year', 'week', 'weekday'],
    # Monthly
    'M': ['year', 'month'],
    'SM': ['year', 'month', 'day'],
    'BM': ['year', 'month'],
    'CBM': ['year', 'month'],
    'MS': ['year', 'month'],
    'SMS': ['year', 'month', 'day'],
    'BMS': ['year', 'month'],
    'CBMS': ['year', 'month'],
    # Quarterly
    'Q': ['year', 'quarter'],
    'BQ': ['year', 'quarter'],
    'QS': ['year', 'quarter'],
    'BQS': ['year', 'quarter'],
    # Yearly
    'A': ['year'],
    'Y': ['year'],
    'BA': ['year'],
    'BY': ['year'],
    'AS': ['year'],
    'YS': ['year'],
    'BAS': ['year'],
    'BYS': ['year'],
    # Hourly
    'BH': ['year', 'month', 'day', 'hour', 'weekday'],
    'H': ['year', 'month', 'day', 'hour'],
    # Minutely
    'T': ['year', 'month', 'day', 'hour', 'minute'],
    'min': ['year', 'month', 'day', 'hour', 'minute'],
    # Secondly
    'S': ['year', 'month', 'day', 'hour', 'minute', 'second'],
    # Milliseconds
    'L': ['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond'],
    'ms': ['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond'],
    # Microseconds
    'U': ['year', 'month', 'day', 'hour', 'minute', 'second', 'microsecond'],
    'us': ['year', 'month', 'day', 'hour', 'minute', 'second', 'microsecond'],
    # Nanoseconds
    'N': []
}

In [ ]:
#| export
class NixtlaClient:

    def __init__(
        self,
        api_key: Optional[str] = None,
        base_url: Optional[str] = None,
        timeout: int = 60,
        max_retries: int = 6,
    ):
        if api_key is None:
            api_key = os.environ['NIXTLA_API_KEY']
        if base_url is None:
            base_url = os.getenv('NIXTLA_BASE_URL', 'https://dashboard.nixtla.io/api')
        self._client_kwargs = {
            'base_url': base_url,
            'headers': {'Authorization': f'Bearer {api_key}'},
            'timeout': timeout,
        }
        self.max_retries = max_retries
        self._model_params: Dict[Tuple[str, str], Tuple[int, int]] = {}

    def _make_request(self, client: HttpClient, endpoint: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        resp = client.request(
            method='post',
            url=endpoint,
            json=payload,
            max_retries=self.max_retries,
        )
        resp_body = resp.json()
        if resp.status_code != 200:
            raise ApiError(status_code=resp.status_code, body=resp_body)
        if 'data' in resp_body:
            resp_body = resp_body['data']
        return resp_body

    def _make_partitioned_requests(
        self,
        client: HttpClient,
        endpoint: str,
        payloads: List[Dict[str, Any]],
    ) -> Dict[str, Any]:
        from tqdm.auto import tqdm

        num_partitions = len(payloads)
        results = num_partitions * [None]
        max_workers = min(10, num_partitions)
        with ThreadPoolExecutor(max_workers) as executor:
            future2pos = {
                executor.submit(self._make_request, client, endpoint, payload): i
                for i, payload in enumerate(payloads)
            }
            for future in tqdm(as_completed(future2pos), total=len(future2pos)):
                pos = future2pos[future]
                results[pos] = future.result()
        resp = {"mean": list(chain.from_iterable(res["mean"] for res in results))}
        first_res = results[0]
        if first_res["intervals"] is None:
            resp["intervals"] = None
        else:
            resp["intervals"] = {}
            for k in first_res["intervals"].keys():
                resp["intervals"][k] = list(
                    chain.from_iterable(res["intervals"][k] for res in results)
                )
        if first_res["weights_x"] is None:
            resp["weights_x"] = None
        else:
            resp["weights_x"] = [res["weights_x"] for res in results]
        return resp

    def _get_model_params(self, model: str, freq: str) -> Tuple[int, int]:
        key = (model, freq)
        if key not in self._model_params:
            logger.info('Querying model metadata...')
            payload = {'model': model, 'freq': freq}
            with httpx.Client(**self._client_kwargs) as httpx_client:
                client = HttpClient(httpx_client=httpx_client)
                params = self._make_request(client, '/model_params', payload)['detail']
            self._model_params[key] = (params['input_size'], params['horizon'])
        return self._model_params[key]

    @staticmethod
    def _standardize_freq(freq: str) -> str:
        return freq.replace('mo', 'MS')

    @staticmethod
    def _tail(proc: ufp.ProcessedDF, n: int) -> ufp.ProcessedDF:
        n_series = proc.indptr.size - 1
        new_sizes = np.minimum(np.diff(proc.indptr), n)
        new_indptr = np.append(0, new_sizes.cumsum())
        new_data = np.empty_like(proc.data, shape=(new_indptr[-1], proc.data.shape[1]))
        for i in range(n_series):
            new_data[new_indptr[i] : new_indptr[i + 1]] = proc.data[
                proc.indptr[i + 1] - new_sizes[i] : proc.indptr[i + 1]
            ]
        return ufp.ProcessedDF(
            uids=proc.uids,
            last_times=proc.last_times,
            data=new_data,
            indptr=new_indptr,
            sort_idxs=None,
        )

    @staticmethod
    def _partition_series(
        payload: Dict[str, Any], n_part: int, h: int
    ) -> List[Dict[str, Any]]:
        parts = []
        series = payload.pop("series")
        n_series = len(series["sizes"])
        n_part = min(n_part, n_series)
        series_per_part = math.ceil(n_series / n_part)
        prev_size = 0
        for i in range(0, n_series, series_per_part):
            sizes = series["sizes"][i : i + series_per_part]
            curr_size = sum(sizes)
            part_idxs = slice(prev_size, prev_size + curr_size)
            prev_size += curr_size
            part_series = {
                "y": series["y"][part_idxs],
                "sizes": sizes,
            }
            if series["X"] is None:
                part_series["X"] = None
                part_series["X_future"] = None
            else:
                part_series["X"] = [x[part_idxs] for x in series["X"]]
                part_series["X_future"] = [
                    x[i * h : (i + series_per_part) * h] for x in series["X_future"]
                ]
            parts.append({"series": part_series, **payload})
        return parts

    @staticmethod
    def _maybe_add_date_features(
        df: DataFrame,
        X_df: Optional[DataFrame],
        features: Union[bool, List[Union[str, Callable]]],
        one_hot: Union[bool, List[str]],
        freq: str,
        h: int,
        id_col: str,
        time_col: str,
        target_col: str,
    ) -> Tuple[DataFrame, Optional[DataFrame]]:
        if not features:
            return df, X_df
        if isinstance(features, list):
            date_features = features
        else:
            date_features = _date_features_by_freq.get(freq, [])
            if not date_features:
                warnings.warn(
                    f'Non default date features for {freq} '
                    'please provide a list of date features'
                )
        # add features
        if X_df is None:
            df, X_df = time_features(
                df=df,
                freq=freq,
                features=date_features,
                h=h,
                id_col=id_col,
                time_col=time_col,
            )
        else:
            df = _add_time_features(df, features=date_features, time_col=time_col)
            X_df = _add_time_features(X_df, features=date_features,time_col=time_col)
        # one hot
        if isinstance(one_hot, list):
            features_one_hot = one_hot
        elif one_hot:
            features_one_hot = [f for f in date_features if not callable(f)]
        else:
            features_one_hot = []
        if features_one_hot:
            X_df = ufp.assign_columns(X_df, target_col, 0)
            full_df = ufp.vertical_concat([df, X_df])
            if isinstance(full_df, pd.DataFrame):
                full_df = pd.get_dummies(
                    full_df, columns=features_one_hot, dtype='float32'
                )
            else:
                full_df = full_df.to_dummies(columns=features_one_hot)
            df = ufp.take_rows(full_df, slice(0, df.shape[0]))
            X_df = ufp.take_rows(full_df, slice(df.shape[0], full_df.shape[0]))
            X_df = ufp.drop_columns(X_df, target_col)
        return df, X_df

    @staticmethod
    def _validate_exog(
        df: DataFrame,
        X_df: Optional[DataFrame],
        id_col: str,
        time_col: str,
        target_col: str,
    ) -> Optional[List[str]]:
        exogs_df = [c for c in df.columns if c not in (id_col, time_col, target_col)]
        if X_df is not None:
            exogs_X = [c for c in X_df.columns if c not in (id_col, time_col)]
            missing_df = set(exogs_X) - set(exogs_df)
            if missing_df:
                raise ValueError(
                    'The following exogenous features are present in `X_df` '
                    f'but not in `df`: {missing_df}.'
                )
            missing_X_df = set(exogs_df) - set(exogs_X)
            if missing_X_df:
                raise ValueError(
                    'The following exogenous features are present in `df` '
                    f'but not in `X_df`: {missing_X_df}.'
                )
            if exogs_df != exogs_X:
                # rearrange columns if necessary
                X_df = X_df[[id_col, time_col, *exogs_df]]
            x_cols: Optional[List[str]] = exogs_df
        else:
            if exogs_df:
                warnings.warn(
                    f'`df` contains the following exogenous features: {exogs_df}, '
                    'but `X_df` was not provided. They will be ignored.'
                )
                df = df[[id_col, time_col, target_col]]
            x_cols = None
            X_df = None
        return df, X_df, x_cols

    @staticmethod
    def _validate_input_size(
        df: DataFrame,
        id_col: str,
        model_input_size: int,
        model_horizon: int,
    ) -> None:
        min_size = ufp.counts_by_id(df, id_col)['counts'].min()
        if min_size < model_input_size + model_horizon:
            raise ValueError(
                'Your time series data is too short '
                'Please make sure that your each serie contains '
                f'at least {model_input_size + model_horizon} observations.'
            )

    @staticmethod
    def _prepare_level_and_quantiles(
        level: Optional[List[Union[int, float]]], 
        quantiles: Optional[List[float]],
    ) -> Tuple[List[Union[int, float]], Optional[List[float]]]:
        if level is not None and quantiles is not None:
            raise ValueError(
                "You should provide `level` or `quantiles`, but not both."
            )
        if quantiles is None:
            return level, quantiles
        # we recover level from quantiles
        if not all(0 < q < 1 for q in quantiles):
            raise ValueError("`quantiles` should be floats between 0 and 1.")
        level = [abs(int(100 - 200 * q)) for q in quantiles]
        return level, quantiles

    @staticmethod
    def _maybe_convert_level_to_quantiles(
        df: DataFrame,
        quantiles: Optional[List[float]],
    ) -> DataFrame:
        if quantiles is None:
            return df
        out_cols = [c for c in df.columns if '-lo-' not in col and '-hi-' not in col]
        df = ufp.copy_if_pandas(df, deep=False)
        for q in sorted(quantiles):
            if q == 0.5:
                col = 'TimeGPT'
            else:
                lv = int(100 - 200 * q)
                hi_or_lo = 'lo' if lv > 0 else 'hi'
                lv = abs(lv)
                col = f"TimeGPT-{hi_or_lo}-{lv}"
            q_col = f"TimeGPT-q-{int(q * 100)}"
            df[q_col] = df[col]
            out_cols.append(q_col)
        return df[out_cols]

    def forecast(
        self,
        df: DataFrame,
        h: PositiveInt,
        freq: str,    
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        X_df: Optional[DataFrame] = None,
        level: Optional[List[Union[int, float]]] = None,
        quantiles: Optional[List[float]] = None,
        finetune_steps: NonNegativeInt = 0,
        finetune_loss: _LOSS = 'default',
        clean_ex_first: bool = True,
        add_history: bool = False,
        date_features: Union[bool, List[Union[str, Callable]]] = False,
        date_features_to_one_hot: Union[bool, List[str]] = True,
        model: _MODEL = 'timegpt-1',
        num_partitions: Optional[int] = None,
    ):
        """Forecast your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we 
                wish to predict or analyze.
            Additionally, you can pass multiple time series (stacked in the dataframe) considering an additional column:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
        h : int
            Forecast horizon.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        X_df : pandas.DataFrame, optional (default=None)
            DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous.
        level : List[float], optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        quantiles : List[float], optional (default=None)
            Quantiles to forecast, list between (0, 1).
            `level` and `quantiles` should not be used simultaneously.
            The output dataframe will have the quantile columns
            formatted as TimeGPT-q-(100 * q) for each q.
            100 * q represents percentiles but we choose this notation
            to avoid having dots in column names.
        finetune_steps : int (default=0)
            Number of steps used to finetune learning TimeGPT in the
            new data.
        finetune_loss : str (default='default')
            Loss function to use for finetuning. Options are: `default`, `mae`, `mse`, `rmse`, `mape`, and `smape`.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        validate_api_key : bool (default=False)
            If True, validates api_key before 
            sending requests.
        add_history : bool (default=False)
            Return fitted values of the model.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        date_features_to_one_hot : bool or list of str (default=True)
            Apply one-hot encoding to these date features.
            If `date_features=True`, then all date features are
            one-hot encoded by default.
        model : str (default='timegpt-1')
            Model to use as a string. Options are: `timegpt-1`, and `timegpt-1-long-horizon`. 
            We recommend using `timegpt-1-long-horizon` for forecasting 
            if you want to predict more than one seasonal 
            period given the frequency of your data.
        num_partitions : int (default=None)
            Number of partitions to use.
            If None, the number of partitions will be equal
            to the available parallel resources in distributed environments.
        
        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with TimeGPT forecasts for point predictions and probabilistic
            predictions (if level is not None).
        """
        self.__dict__.pop('weights_x', None)
        logger.info('Validating inputs...')
        validate_format(df=df, id_col=id_col, time_col=time_col, target_col=target_col)
        if ufp.is_nan_or_none(df[target_col]).any():
            raise ValueError(f'Target column ({target_col}) cannot contain missing values.')
        validate_freq(times=df[time_col], freq=freq)
        df, X_df, x_cols = self._validate_exog(
            df, X_df, id_col=id_col, time_col=time_col, target_col=target_col
        )
        level, quantiles = self._prepare_level_and_quantiles(level, quantiles)
        standard_freq = self._standardize_freq(freq)
        model_input_size, model_horizon = self._get_model_params(model, standard_freq)
        if finetune_steps > 0 or level is not None:
            self._validate_input_size(df, id_col, model_input_size, model_horizon)

        logger.info('Preprocessing dataframes...')
        df, X_df = self._maybe_add_date_features(
            df=df,
            X_df=X_df,
            features=date_features,
            one_hot=date_features_to_one_hot,
            freq=standard_freq,
            h=h,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
        )
        processed = ufp.process_df(
            df=df, id_col=id_col, time_col=time_col, target_col=target_col
        )
        if X_df is not None:
            processed_X = ufp.process_df(
                df=X_df, id_col=id_col, time_col=time_col, target_col=None,
            )
            X_future = processed_X.data.T.tolist()
        else:
            X_future = None

        if h > model_horizon:
            logger.warning(
                'The specified horizon "h" exceeds the model horizon. '
                'This may lead to less accurate forecasts. '
                'Please consider using a smaller horizon.'  
            )
        restrict_input = finetune_steps == 0 and X_df is None and not add_history
        if restrict_input:
            logger.info('Restricting input...')
            new_input_size = _restrict_input_samples(
                level=level,
                input_size=model_input_size,
                model_horizon=model_horizon,
                h=h,
            )
            processed = self._tail(processed, new_input_size)
        if processed.data.shape[1] > 1:
            X = processed.data[:, 1:].T.tolist()
        else:
            X = None
        payload = {
            'series': {
                'y': processed.data[:, 0].tolist(),
                'sizes': np.diff(processed.indptr).tolist(),
                'X': X,
                'X_future': X_future,
            },
            'model': model,
            'h': h,
            'freq': standard_freq,
            'clean_ex_first': clean_ex_first,
            'level': level,
            'finetune_steps': finetune_steps,
            'finetune_loss': finetune_loss,
        }

        logger.info('Calling Forecast Endpoint...')
        with httpx.Client(**self._client_kwargs) as httpx_client:
            client = HttpClient(httpx_client=httpx_client)
            if num_partitions is None:
                resp = self._make_request(client, '/v2/forecast', payload)
            else:
                payloads = self._partition_series(payload, num_partitions, h)
                resp = self._make_partitioned_requests(client, '/v2/forecast', payloads)

        # assemble result
        out = ufp.make_future_dataframe(
            uids=processed.uids,
            last_times=type(processed.uids)(processed.last_times),
            freq=freq,
            h=h,
            id_col=id_col,
            time_col=time_col,
        )
        out = ufp.assign_columns(out, 'TimeGPT', resp['mean'])
        if resp['intervals'] is not None:
            intervals_df = type(df)(
                {
                    f'TimeGPT-{k}': v for k, v in resp['intervals'].items()
                }
            )
            intervals_df = self._maybe_convert_level_to_quantiles(
                intervals_df, quantiles
            )
            out = ufp.horizontal_concat([out, intervals_df])
        if resp['weights_x'] is not None:
            self.weights_x = type(df)({
                'features': x_cols,
                'weights': resp['weights_x'],
            })
        return out

In [ ]:
import polars as pl
from fastcore.test import test_warns
from utilsforecast.data import generate_series

from nixtla import NixtlaClient as NixtlaV1

In [ ]:
series = generate_series(
    4, min_length=40, max_length=200, n_static_features=2, static_as_categorical=False
)
horizon = 20
level = [80, 97.5]
series['unique_id'] = series['unique_id'].astype('int64')
valid = series.groupby('unique_id', observed=True).tail(horizon)
X_df = valid.drop(columns='y')
train = series.drop(valid.index)
train_pl = pl.from_pandas(train)
X_df_pl = pl.from_pandas(X_df)

In [ ]:
client = NixtlaClient()

In [ ]:
resv2_pd = client.forecast(df=train, X_df=X_df, h=horizon, freq="D", level=level)

In [ ]:
test_warns(lambda: client.forecast(df=train, h=1, freq="D", level=level))

In [ ]:
resv2_pl = client.forecast(df=train_pl, X_df=X_df_pl, h=horizon, freq="1d", level=level)

In [ ]:
np.testing.assert_allclose(
    client.forecast(df=series[['unique_id', 'ds', 'y']], h=horizon, freq="D")['TimeGPT'].to_numpy(),
    client.forecast(df=series[['unique_id', 'ds', 'y']], h=horizon, freq="D", num_partitions=4)['TimeGPT'].to_numpy(),
    rtol=1e-6,
)

In [ ]:
pd.testing.assert_frame_equal(resv2_pd, resv2_pl.to_pandas())

In [ ]:
v1_client = NixtlaV1()
resv1 = v1_client.forecast(df=train, X_df=X_df, h=horizon, freq="D", level=level)

In [ ]:
pd.testing.assert_frame_equal(resv2_pd.astype({'ds': 'str'}), resv1)